# Extracting the data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re  
import nltk
import string
from bs4 import BeautifulSoup
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from textblob import TextBlob

In [2]:
import yaml

config = dict(
    search_tweets_api = dict(
        account_type = 'premium',
        endpoint = 'https://api.twitter.com/1.1/tweets/search/30day/KitchenAid.json',
        consumer_key = 'foyCkjGvs1JSBVW3iyoffhITm',
        consumer_secret = 'aeDuwdyFt86cFfN8sia8tHkiIExSsQjwvusdetk5AguUojnG2z'
    )
)

with open('twitter_keys_30days.yaml', 'w') as config_file:
    yaml.dump(config, config_file, default_flow_style=False)

In [3]:
from searchtweets import load_credentials

premium_search_args = load_credentials("twitter_keys_30days.yaml",
                                       yaml_key="search_tweets_api",
                                       env_overwrite=False)
print(premium_search_args)

C:\Users\Acer\Anaconda3\lib\site-packages\searchtweets\credentials.py:34: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  search_creds = yaml.load(f)[yaml_key]
Grabbing bearer token from OAUTH


{'bearer_token': 'AAAAAAAAAAAAAAAAAAAAAEamAwEAAAAAq91t%2BQg1%2BH5aPOp254JFGXeyRas%3Do2eu2ThehePIfHSE5ZUI3hExxJdqFsG9VtltO4XJ3m9On7dqGf', 'endpoint': 'https://api.twitter.com/1.1/tweets/search/30day/KitchenAid.json', 'extra_headers_dict': None}


In [4]:
from searchtweets import gen_rule_payload

rule = gen_rule_payload("kitchenaid", 
                        results_per_call=100#,
                        #from_date="2019-11-01 07:15",
                        #to_date="2019-11-31 23:11"
                       )

In [5]:
from searchtweets import ResultStream

rs = ResultStream(rule_payload=rule,
                  max_results=500,
                  **premium_search_args)
print(rs)

ResultStream: 
	{
    "username": null,
    "endpoint": "https://api.twitter.com/1.1/tweets/search/30day/KitchenAid.json",
    "rule_payload": {
        "query": "kitchenaid",
        "maxResults": 100
    },
    "tweetify": true,
    "max_results": 500
}


In [6]:
import json

with open('test1.jsonl', 'a', encoding='utf-8') as f:
    n = 0
    for tweet in rs.stream():
        n += 1
        if n % 10 == 0:
            print('{0}: {1}'.format(str(n), tweet['created_at']))
        json.dump(tweet, f)
        f.write('\n')
print('done')

10: Thu Dec 19 19:55:16 +0000 2019
20: Thu Dec 19 19:10:14 +0000 2019
30: Thu Dec 19 18:17:29 +0000 2019
40: Thu Dec 19 17:31:03 +0000 2019
50: Thu Dec 19 16:57:02 +0000 2019
60: Thu Dec 19 16:34:49 +0000 2019
70: Thu Dec 19 15:48:48 +0000 2019
80: Thu Dec 19 14:45:36 +0000 2019
90: Thu Dec 19 14:14:14 +0000 2019
100: Thu Dec 19 13:28:08 +0000 2019
110: Thu Dec 19 12:47:52 +0000 2019
120: Thu Dec 19 11:38:39 +0000 2019
130: Thu Dec 19 10:30:00 +0000 2019
140: Thu Dec 19 08:59:11 +0000 2019
150: Thu Dec 19 08:11:04 +0000 2019
160: Thu Dec 19 06:05:57 +0000 2019
170: Thu Dec 19 04:58:37 +0000 2019
180: Thu Dec 19 03:51:12 +0000 2019
190: Thu Dec 19 03:08:38 +0000 2019
200: Thu Dec 19 02:10:47 +0000 2019
210: Thu Dec 19 01:28:39 +0000 2019
220: Thu Dec 19 00:34:19 +0000 2019
230: Wed Dec 18 23:36:33 +0000 2019
240: Wed Dec 18 22:32:04 +0000 2019
250: Wed Dec 18 21:43:54 +0000 2019
260: Wed Dec 18 20:59:24 +0000 2019
270: Wed Dec 18 20:27:42 +0000 2019
280: Wed Dec 18 20:02:44 +0000 2019
2

# Load the data

In [7]:
import json_lines

def load_jsonl(file):
    tweets = []
    with open(file, 'rb') as f:
        for tweet in json_lines.reader(f, broken=True):
            reduced_tweet = {
                'username' : tweet['user']['screen_name'],
                'status_text' : tweet['text'],
                'hashtags' : tweet['entities']['hashtags'],
                
                }
                
            tweets.append(reduced_tweet)
    return (tweets)

tweets = load_jsonl('test1.jsonl')

In [8]:
tweets[1]

{'username': 'richardson_ceri',
 'status_text': 'RT @WithFranca: ✨INSTAGRAM GIVEAWAY✨ Our Big #Giveaway is now live! “WIN a KitchenAid Artisan Stand Mixer 4.8L bowl in Matte Grey worth a w…',
 'hashtags': [{'text': 'Giveaway', 'indices': [45, 54]}]}

In [9]:
import re
import json

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
regex_str = [
    emoticons_str,
    r'<[^>]+>',  # HTML Tags
    r'(?:@[\w_]+)',  # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)",  # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',  # URLs
    r'(?:(?:\d+,?)+(?:\.?\d+)?)',  # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])",  # words with - and '
    r'(?:[\w_]+)',  # other words
    r'(?:\S)' # anything else
    
]

tokens_re = re.compile(r'(' + '|'.join(regex_str) + ')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^' + emoticons_str + '$', re.VERBOSE | re.IGNORECASE)


def tokenize(s):
    return tokens_re.findall(s)


def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [10]:
with open("test1.jsonl", "r") as f:
    lines = f.readlines()
with open("test1.jsonl", "w") as f:
    for line in lines:
        if line.strip("\n") != "nickname_to_delete":
            f.write(line)

In [11]:
with open('test1.jsonl', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        tokens = preprocess(tweet['text'])

In [12]:
tweets[1]

{'username': 'richardson_ceri',
 'status_text': 'RT @WithFranca: ✨INSTAGRAM GIVEAWAY✨ Our Big #Giveaway is now live! “WIN a KitchenAid Artisan Stand Mixer 4.8L bowl in Matte Grey worth a w…',
 'hashtags': [{'text': 'Giveaway', 'indices': [45, 54]}]}

In [13]:
import operator 
import json
from collections import Counter
 
fname = 'test1.jsonl'
with open(fname, 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        # Create a list with all the terms
        terms_all = [term for term in preprocess(tweet['text'])]
        # Update the counter
        count_all.update(terms_all)
    # Print the first 5 most frequent words
    print(count_all.most_common(10))

[('KitchenAid', 2047), ('a', 1943), ('…', 1921), ('.', 1900), (':', 1878), ('!', 1813), ('-', 1356), ('RT', 1206), ('the', 1118), ('to', 1092)]


In [14]:
from nltk.corpus import stopwords
import string
 
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']

In [15]:
with open('test1.jsonl', 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        # Create a list with all the terms
        terms_stop = [term for term in preprocess(tweet['text']) if term not in stop]
        # Update the counter
        count_all.update(terms_stop)
# Print the first 5 most frequent words
print(count_all.most_common(10))

[('KitchenAid', 2047), ('…', 1921), ('RT', 1206), ('Sue', 1064), ('Mixer', 712), ('✨', 648), ('Artisan', 563), ('Stand', 548), ('Toronto', 532), ('attack', 532)]


In [16]:
print(count_all.most_common(10))
fname = 'test1.jsonl'
with open(fname, 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
       # Count terms only once, equivalent to Document Frequency
    terms_single = set(terms_all)
    # Count hashtags only
    terms_hash = [term for term in preprocess(tweet['text']) 
                  if term.startswith('#')]
    # Update the counter
    count_all.update(terms_hash)
    # Print the first 5 most frequent words
    print(count_all.most_common(10))

[('KitchenAid', 2047), ('…', 1921), ('RT', 1206), ('Sue', 1064), ('Mixer', 712), ('✨', 648), ('Artisan', 563), ('Stand', 548), ('Toronto', 532), ('attack', 532)]
[]


In [17]:
terms_single = set(terms_all)
# Count terms only (no hashtags, no mentions)
terms_only = [term for term in preprocess(tweet['text']) 
              if term not in stop and
              not term.startswith(('#', '@'))] 
              # mind the ((double brackets))
              # startswith() takes a tuple (not a list) if 
              # we pass a list of inputs
count_all.update(terms_only)
print(count_all.most_common(10))

[('Sue', 4), ('Toronto', 2), ('attack', 2), ('RT', 1), ("Let's", 1), ('Play', 1), ('Innocent', 1), ('Smith', 1), ('amp', 1), ('Wesson', 1)]


In [19]:
from nltk import bigrams 
 
terms_bigram = bigrams(terms_stop)